# Objective:
* Translate all product models into plain English
* Convert local price to USD using the current foreign exchange rate
* Find the cheapest prices of each product model

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_datareader import data
import re
import plotly.express as px

import ipywidgets as widgets
from ipywidgets import interact

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# read data

In [1]:
df=pd.read_csv('../input/apple-product-price-list-from-26-countries-2022/apple product price list from 26 countries.csv')
df['country_code']=df['country_code'].str.upper()
df

# translate all foreign model names into English name (model_en column)

In [1]:
df['model_en']=df['model']

df.loc[df['model'].str.contains('imac', case=False), 'model_en'] = '24-inch iMac'
df.loc[df['model'].str.contains('pen', case=False), 'model_en'] = 'Apple Pencil (2nd generation)'
df.loc[df['model'].str.contains('spor|Спортивный ремешок|Correa deportiva|Urheiluranneke', case=False), 'model_en'] = 'Sport Band'
df.loc[df['model'].str.contains('airpods\(2|airpods\(andra|airpods\(รุ่นที่ 2|airpods\(seconda|airpods\(segunda', case=False), 'model_en'] = 'AirPods(2nd generation)'
df.loc[df['model'].str.contains('airpods\(3|airpods\(รุ่นที่ 3|airpods\(tre|airpods\(ter', case=False), 'model_en'] = 'AirPods(3rd generation)'
df.loc[df['model'].str.contains('mouse', case=False), 'model_en'] = 'Magic Mouse'
df.loc[df['model'].str.contains('Apple Store|iTunes', case=False), 'model_en'] = 'App Store and iTunes Gift Card'


In [1]:
df['model_en'].value_counts()

# extract (starting) price

In [1]:
df['price2']=df['price'].str.split('or', 1, expand=True)[0]
df

In [1]:
# keep only digits, . (period), and , (comma)
df['price2']=df['price2'].str.replace('[^\d\.\,]', '', regex=True)
df

### take care of different decimal separators

In [1]:
# credit: https://stackoverflow.com/questions/43211380/is-there-a-list-of-locale-codes-of-countries-that-use-commas-instead-of-decimal
comma_decimal_point_locale = ['AM', 'AR', 'AT', 'AZ', 'BA', 'BE', 'BG', 'BO', 'BR', 'BY',
                    'CA', 'CH', 'CL', 'CM', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK',
                    'EC', 'EE', 'ES', 'FI', 'FO', 'FR', 'GE', 'GL',  'HR', 'HU',
                    'ID', 'IS', 'IT', 'KZ', 'LB', 'LT', 'LU', 'LV', 'MA', 'MD', 'MK', 'MO', 'MZ',
                    'NL', 'NO', 'PE', 'PL', 'PT', 'PY', 'RO', 'RS', 'RU',
                    'SE', 'SI', 'SK', 'TN', 'TR', 'UA', 'UY', 'UZ', 'VE', 'VN', 'ZA']

price_clean=[]
for i, r in df.iterrows():
    p=r['price2']
    if r['country_code'] in comma_decimal_point_locale:
        p=p.replace('.', '').replace(',', '.')
    else:
        p=p.replace(',', '')
    price_clean.append(p)
    
df['price_local_currency']=price_clean
df['price_local_currency']=pd.to_numeric(df['price_local_currency'])
df

# Interactive dataframe (filter by country name)

In [1]:
df['country'].unique()

In [1]:
@interact
def filter(regex_pat="."):
    return df[(df["country"].str.contains(regex_pat, regex=True, case=False))].sort_values(
        "pid", ascending=True
    )

# get Exchange rate

In [1]:
dc=pd.read_csv('https://raw.githubusercontent.com/prasertcbs/basic-dataset/master/country%20and%20currency%20code.csv')
dc=dc.drop_duplicates(subset=['Alpha2 Code']).reset_index(drop=True)
dc

In [1]:
df=pd.merge(df, dc[['Alpha2 Code', 'Currency CodeA']], how='inner', left_on='country_code', right_on='Alpha2 Code').drop(columns=['Alpha2 Code'])
df

## get the latest foreign exchange rate

In [1]:
# (pd.Timestamp.now() + pd.Timedelta(days=-5)).strftime('%Y-%m-%d')

dfx=data.DataReader(df['Currency CodeA'].unique() + '=X', 'yahoo', 
                    start=(pd.Timestamp.now() + pd.Timedelta(days=-5)).strftime('%Y-%m-%d'), end=pd.Timestamp.now().strftime('%Y-%m-%d'))['Close'].stack()
dfx=dfx.to_frame().reset_index()
dfx.columns=['FX_Date', 'Currency CodeA', 'X']
dfx['Currency CodeA']=dfx['Currency CodeA'].str[:3]
dfx=dfx.drop_duplicates(subset=['Currency CodeA'], keep='last')
dfx

In [1]:
# dfx.to_csv('exchange rate.csv', index=False)

In [1]:
df=pd.merge(df, dfx[['Currency CodeA', 'X']], how='outer', left_on='Currency CodeA', right_on='Currency CodeA')
df['price_usd']=df['price_local_currency']/df['X']
df

In [1]:
# df.to_csv('clean.csv', index=False)

# Interactive product price comparison across country

In [1]:
models=sorted(df['model_en'].unique().tolist(), key=str.lower)
# models

@interact
def filter(model_list=models):
    d_filter=df[(df["model_en"]==model_list)].sort_values("price_usd", ascending=True)
    d_filter=d_filter[d_filter['price_usd'].notna()]
    return d_filter.drop(columns=['pid', 'country_code', 'scraped_date']).style.bar(subset=['price_usd'], color='deepskyblue')

# Cheapest price of each product model

In [1]:
grp=df.groupby('model_en')
lowest_price=grp.apply(lambda g: g.nsmallest(1, 'price_usd'))[['country', 'price_usd']]
lowest_price=lowest_price.reset_index().drop(columns=['level_1'])
avg_price=df.groupby('model_en')['price_usd'].agg(['mean', 'max']).reset_index()
avg_price.rename(columns={'mean': 'avg_price_usd', 'max': 'max_price_usd'}, inplace=True)
# avg_price
pd.merge(lowest_price, avg_price, left_on=['model_en'], right_on=['model_en'])

# turn into wide format

In [1]:
pvt=pd.pivot_table(df, index=['model_en'], columns=['country'], values=['price_usd'], )
pvt

In [1]:
pvt.loc[:, 'price_usd']

In [1]:
pvt.loc[:, 'price_usd'].to_csv('apple product prices wide format.csv')

In [1]:
df